# Venture Funding with Deep Learning
This project uses TensorFlow and Keras to create a binary classification model that determines whether a startup should receive funding from a venture capital firm. I will be testing 3 different neural network models to see which one performs the best. For this project, I was given the following scenario:

>You work as a risk management associate at Alphabet Soup, a venture capital firm. Alphabet Soup’s business team receives many funding applications from startups every day. This team has asked you to help them create a model that predicts whether applicants will be successful if funded by Alphabet Soup. The business team has given you a CSV containing more than 34,000 organizations that have received funding from Alphabet Soup over the years. With your knowledge of machine learning and neural networks, you decide to use the features in the provided dataset to create a binary classifier model that will predict whether an applicant will become a successful business. The CSV file contains a variety of information about these businesses, including whether or not they ultimately became successful.

In [359]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

## Step 1: Prepare the data to be used on a neural network model

### Step 1.1: Read the `applicants_data.csv` file into a Pandas DataFrame. Review the DataFrame, looking for categorical variables that will need to be encoded, as well as columns that could eventually define your features and target variables.  

In [360]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv(
    Path('./Resources/applicants_data.csv')
)

# Review the DataFrame
applicant_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [361]:
# Review the data types associated with the columns
applicant_data_df.dtypes


EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

### Step 1.2: Drop the “EIN” (Employer Identification Number) and “NAME” columns from the DataFrame, because they are not relevant to the binary classification model.

In [308]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=['EIN', 'NAME'])

# Review the DataFrame
applicant_data_df


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


### Step 1.3: Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

In [309]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes == 'object'].index)

# Display the categorical variables list
categorical_variables

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [310]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [311]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])


In [312]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns=enc.get_feature_names_out(categorical_variables)
)

# Review the DataFrame
encoded_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 1.4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

In [313]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
numerical_df = applicant_data_df.drop(columns=categorical_variables)

encoded_df = pd.concat([encoded_df, numerical_df], axis=1)

# Review the Dataframe
encoded_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT,IS_SUCCESSFUL
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,108590,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6692,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,142590,1


### Step 1.5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 

In [314]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df['IS_SUCCESSFUL']

# Display a sample of y
y[:5]


0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [315]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns='IS_SUCCESSFUL')

# Review the features DataFrame
X.head()


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,108590
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6692
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,142590


### Step 1.6: Split the features and target sets into training and testing datasets.

In [316]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


### Step 1.7: Use scikit-learn's `StandardScaler` to scale the features data.

In [317]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Step 2: Compile and Evaluate a Binary Classification Model Using a Neural Network

### Step 2.1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

In [362]:
# Define the the number of inputs (features) to the model
number_input_features = 116

# Review the number of features
number_input_features


116

In [319]:
# Define the number of neurons in the output layer
number_output_neurons = 1

To determine the number of hidden nodes for the first layer, I used the mean of the inputs and outputs. For the second layer, I divided the number of nodes in the first layer by 2.

In [363]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  round((number_input_features + number_output_neurons) / 2)

# Review the number hidden nodes in the first layer
hidden_nodes_layer1


58

In [366]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  hidden_nodes_layer1 / 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2


29.0

In [322]:
# Create the Sequential model instance
nn = Sequential()


In [323]:
# Add the first hidden layer
nn.add(Dense(
    units=hidden_nodes_layer1,
    input_dim=number_input_features,
    activation='relu'
))

In [324]:
# Add the second hidden layer
nn.add(Dense(
    units=hidden_nodes_layer2,
    activation='relu'
))

In [325]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(
    units=number_output_neurons,
    activation='sigmoid'
))

In [326]:
# Display the Sequential model summary
print(nn.summary())


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_34 (Dense)            (None, 58)                6786      
                                                                 
 dense_35 (Dense)            (None, 29)                1711      
                                                                 
 dense_36 (Dense)            (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________
None


### Step 2.2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.

In [327]:
# Compile the Sequential model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [328]:
# Fit the model using 50 epochs and the training data
model_1 = nn.fit(X_train_scaled, y_train, epochs=50, verbose=0)

### Step 2.3: Evaluate the model using the test data to determine the model’s loss and accuracy.

In [329]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5564 - accuracy: 0.7305 - 246ms/epoch - 919us/step
Loss: 0.556383490562439, Accuracy: 0.7304956316947937


### Step 2.4: Save and export your model to an HDF5 file, and name the file `AlphabetSoup.h5`. 

In [330]:
# Set the model's file path
file_path = Path('./Resources/AlphabetSoup.h5')

# Export your model to a HDF5 file
nn.save(file_path)


## Step 3: Optimize the neural network model

### Step 3.1: Define at least three new deep neural network models (resulting in the original plus 3 optimization attempts). With each, try to improve on your first model’s predictive accuracy.

### Alternative Model 1
For this model, I wanted to see if eliminating certain features would impact the accuracy of the model. Specifically, I looked at the 'STATUS' and 'LOSS' features because it seemed like they would not have much of an impact on the overall success of the startup, and I wanted to see if those features had imbalanced classes, which could potentially affect the acuracy of the model.

In [367]:
# print the value counts for the 'STATUS' and 'SPECIAL_CONSIDERATIONS' columns to see if the classes are imbalanced
display(applicant_data_df['STATUS'].value_counts())
display(applicant_data_df['SPECIAL_CONSIDERATIONS'].value_counts())

1    34294
0        5
Name: STATUS, dtype: int64

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [368]:
# Remove special considerations from categorical variables
categorical_variables_A1 = categorical_variables[:6]
categorical_variables_A1

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [369]:
# Encode the categorical variables
encoded_data_A1 = enc.fit_transform(applicant_data_df[categorical_variables_A1])

In [370]:
# Create a DataFrame for the encoded categorical variables
encoded_data_A1_df = pd.DataFrame(
    encoded_data_A1,
    columns=enc.get_feature_names_out(categorical_variables_A1)
)

# Review the DataFrame
encoded_data_A1_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [335]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
numerical_df_A1 = applicant_data_df.drop(columns=categorical_variables)
numerical_df_A1 = numerical_df_A1.drop(columns='STATUS')

encoded_df_A1 = pd.concat([encoded_data_A1_df, numerical_df_A1], axis=1)

# Review the Dataframe
encoded_df_A1.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,ASK_AMT,IS_SUCCESSFUL
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108590,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6692,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,142590,1


In [336]:
# Define the target set y using the IS_SUCCESSFUL column
y_A1 = encoded_df_A1['IS_SUCCESSFUL']

# Display a sample of y
y_A1[:5]


0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [337]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X_A1 = encoded_df_A1.drop(columns='IS_SUCCESSFUL')

# Review the features DataFrame
X_A1.head()


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,ASK_AMT
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108590
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6692
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,142590


In [338]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train_A1, X_test_A1, y_train_A1, y_test_A1 = train_test_split(X_A1, y_A1, random_state=1)

In [339]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler_A1 = scaler.fit(X_train_A1)

# Fit the scaler to the features training dataset
X_train_A1_scaled = X_scaler_A1.transform(X_train_A1)
X_test_A1_scaled = X_scaler_A1.transform(X_test_A1)

In [340]:
# Define the the number of inputs (features) to the model
number_input_features_A1 = len(X_train_A1.iloc[0])

# Review the number of features
number_input_features_A1

113

In [341]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 1

In [342]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = (number_input_features + number_output_neurons_A1)/2

hidden_nodes_layer2_A1 = 29

In [343]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [344]:
# First hidden layer
nn_A1.add(Dense(
    units=hidden_nodes_layer1_A1,
    input_dim=number_input_features_A1,
    activation='relu'
))

# Second hidden layer
nn_A1.add(Dense(
    units=hidden_nodes_layer2_A1,
    activation='relu'
))

# Output layer
nn_A1.add(Dense(
    units=number_output_neurons_A1,
    activation='sigmoid'
))

# Check the structure of the model
print(nn_A1.summary())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_37 (Dense)            (None, 58)                6612      
                                                                 
 dense_38 (Dense)            (None, 29)                1711      
                                                                 
 dense_39 (Dense)            (None, 1)                 30        
                                                                 
Total params: 8,353
Trainable params: 8,353
Non-trainable params: 0
_________________________________________________________________
None


In [345]:
# Compile the Sequential model
nn_A1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [346]:
# Fit the model using 50 epochs and the training data
fit_model_A1 = nn_A1.fit(X_train_A1_scaled, y_train_A1, epochs=50, verbose=0)


#### Alternative Model 2
For the second alternative model, I wanted to see if just having one hidden layer would be the same or better than having 2 hidden layers. This model will take the same input features as the first model.

In [347]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [348]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 1

In [349]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = round((number_input_features + number_output_neurons) / 2)

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A2

58

In [350]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [351]:
# First hidden layer
nn_A2.add(Dense(
    units=hidden_nodes_layer1_A2,
    input_dim=number_input_features,
    activation='relu'
))

# Output layer
nn_A2.add(Dense(
    units=1,
    activation='sigmoid'
))

# Check the structure of the model
print(nn_A2.summary())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 58)                6786      
                                                                 
 dense_41 (Dense)            (None, 1)                 59        
                                                                 
Total params: 6,845
Trainable params: 6,845
Non-trainable params: 0
_________________________________________________________________
None


In [352]:
# Compile the model
nn_A2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [353]:
# Fit the model
nn_A2.fit(X_train_scaled, y_train, epochs=50, verbose=0)

### Step 3.2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.
Comparing the different models, it doesn't seem that the accuracy or loss were impacted much by the changes I made. The accuracy for the model with 1 hidden layer was the highest, but only by a very small amount

In [354]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
268/268 - 0s - loss: 0.5564 - accuracy: 0.7305 - 170ms/epoch - 636us/step
Loss: 0.556383490562439, Accuracy: 0.7304956316947937


In [355]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A1.evaluate(X_test_A1_scaled, y_test_A1, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results
268/268 - 0s - loss: 0.5532 - accuracy: 0.7304 - 244ms/epoch - 911us/step
Loss: 0.5532251596450806, Accuracy: 0.7303789854049683


In [356]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results
268/268 - 0s - loss: 0.5561 - accuracy: 0.7315 - 230ms/epoch - 858us/step
Loss: 0.5561337471008301, Accuracy: 0.7315452098846436


### Step 3.3: Save each of your alternative models as an HDF5 file.

In [357]:
# Set the file path for the first alternative model
file_path = Path('./Resources/AlphabetSoup_A1.h5')

# Export your model to a HDF5 file
nn_A1.save(file_path)

In [358]:
# Set the file path for the second alternative model
file_path = Path('./Resources/AlphabetSoup_A2.h5')

# Export your model to a HDF5 file
nn_A2.save(file_path)
